# Burgers Equation
Equation:   $u_{t} + uu_{x}-\frac{0.01}{\pi}u_{xx} = 0$  
Boundary Conditions:  
$x \in [-1,1]$  $t \in [0,1]$  
$u(0,x)= -\sin(\pi x)$  
$u(t,-1)=u(t,1)=0$ 

## Import Libraries

In [13]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader

In [14]:
!pip install pyDOE    #required for latin hypercube sampling of collocation points
from pyDOE import lhs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Prepocessing

In [15]:
nu = 0.01/np.pi
N_u = 100                                                                       #boundary points
N_f = 10000                                                                     #collacation points
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat')                                    #contains x,t and exact usol
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  #2 columns containing x,t values
u_star = Exact.flatten()[:,None]                                                #1 column containing exact u values 
lb = X_star.min(0)                                                              #lower & upper bounds for x & t
ub = X_star.max(0) 
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)                                            #Latin Hypercube Sampling method to generate collacation points
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)                  #Randomly choosing 100 training points
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

X_u = torch.from_numpy(X_u_train[:,0:1]).float()         #x boundary points 
T_u = torch.from_numpy(X_u_train[:,1:2]).float()         #t boundary points 
X_f = torch.tensor(X_f_train[:,0:1], requires_grad = True).float()        #x collocation points
T_f = torch.tensor(X_f_train[:,1:2], requires_grad = True).float()         #t collocation points
u_train = torch.from_numpy(u_train).float()

In [16]:
class Data(Dataset):

  def __init__(self, X_u, T_u, u_train):
    self.x = torch.cat([X_u, T_u], axis=1).float()
    self.y = u_train
    self.len = u_train.shape[0]
  
  def __getitem__(self, index):    
      return self.x[index], self.y[index]
  
  def __len__(self):
      return self.len

In [17]:
data_set = Data(X_u, T_u, u_train)
train_loader = DataLoader(dataset=data_set, batch_size=50)

## Burger's Equation PINN (Formulation & Implementation)

In [18]:
class PINN(nn.Module):

  def __init__(self, layers):
    super(PINN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x   

In [19]:
def residual_loss(X_f, T_f, model, nu):
  xf = torch.cat([X_f, T_f], axis=1)
  uf = model(xf)
  u_x = grad(uf.sum(), X_f, retain_graph = True, create_graph = True)[0]
  u_xx = grad(u_x.sum(), X_f, retain_graph = True, create_graph = True)[0]
  u_t = grad(uf.sum(), T_f, retain_graph = True, create_graph = True)[0]
  f = u_t + uf*u_x - nu*u_xx 
  return torch.mean(torch.square(f))

In [21]:
def PINN_train(model, train_loader, optimizer, X_f, T_f, nu):
  epochs = 20000
  mse = nn.MSELoss()
  for epoch in range(epochs):
    for x, y in train_loader:
      model.train()
      optimizer.zero_grad()
      yhat = model(x)
      loss1 = mse(yhat, y)
      loss2 = residual_loss(X_f, T_f, model, nu)
      loss = loss1 + loss2
      loss.backward()
      optimizer.step()
    if epoch % 500 == 0:
      print('Epoch:', epoch, 'Loss: %.5e' % (loss.item()))
  return

In [22]:
pinn_model = PINN(layers)
optimizer = torch.optim.Adam(pinn_model.parameters(), lr = 1e-4)

In [23]:
PINN_train(pinn_model, train_loader, optimizer, X_f, T_f, nu)

Epoch: 0 Loss: 2.46112e-01
Epoch: 500 Loss: 8.32914e-02
Epoch: 1000 Loss: 7.17834e-02
Epoch: 1500 Loss: 6.05222e-02
Epoch: 2000 Loss: 5.23553e-02
Epoch: 2500 Loss: 4.56048e-02
Epoch: 3000 Loss: 4.17674e-02
Epoch: 3500 Loss: 3.60842e-02
Epoch: 4000 Loss: 1.25536e-02
Epoch: 4500 Loss: 6.54742e-03
Epoch: 5000 Loss: 4.41096e-03
Epoch: 5500 Loss: 3.42569e-03
Epoch: 6000 Loss: 2.41042e-03
Epoch: 6500 Loss: 2.05730e-03
Epoch: 7000 Loss: 1.83030e-03
Epoch: 7500 Loss: 1.83090e-03
Epoch: 8000 Loss: 1.13324e-03
Epoch: 8500 Loss: 8.94912e-04
Epoch: 9000 Loss: 7.82691e-04
Epoch: 9500 Loss: 7.07575e-04
Epoch: 10000 Loss: 6.21982e-04
Epoch: 10500 Loss: 5.55963e-04
Epoch: 11000 Loss: 5.41247e-04
Epoch: 11500 Loss: 4.82224e-04
Epoch: 12000 Loss: 4.23994e-04
Epoch: 12500 Loss: 4.03897e-04
Epoch: 13000 Loss: 3.71028e-04
Epoch: 13500 Loss: 4.27452e-04
Epoch: 14000 Loss: 3.21217e-04
Epoch: 14500 Loss: 3.82691e-04
Epoch: 15000 Loss: 2.85728e-04
Epoch: 15500 Loss: 3.72869e-04
Epoch: 16000 Loss: 2.64862e-04
E

In [25]:
pinn_model.eval()
u_pinn = pinn_model(torch.from_numpy(X_star).float())
table = np.hstack((u_pinn.detach().numpy(), u_star))
print('Predicted   -   Actual')
print(table[10010:10020])

Predicted   -   Actual
[[0.28493625 0.28533147]
 [0.29575807 0.29615185]
 [0.30656067 0.30695429]
 [0.3173413  0.31773801]
 [0.32810009 0.32850224]
 [0.33883479 0.33924619]
 [0.34954569 0.34996906]
 [0.36023    0.36067002]
 [0.37088776 0.37134826]
 [0.38151738 0.38200292]]
